<a href="https://colab.research.google.com/github/vidushiMaheshwari/SWIN-IR/blob/main/SWIN_IR_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset

In [ ]:
!wget 'https://nihcc.box.com/shared/static/sp5y2k799v4x1x77f7w1aqp26uyfq7qz.zip'
## after this rename the folder for the sake of simplicity (I am using zip_file.zip)

In [ ]:
!wget 'https://nihcc.app.box.com/v/DeepLesion/file/305569842654'
## renamed to csv_file.csv

In [3]:
!unzip -q zip_file.zip -d zip_file

Now that we have images in a folder, let's start with the actual implementation of model

In [47]:
# creating a dataset object
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
import torch
from skimage import io

class CustomImageDataset(Dataset):
  def __init__(self, csv_file, root_dir, transform=None):
    self.img_labels=pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transforms = transform
  def __len__(self):
    return len(self.img_labels)
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx = idx.tolist()
    img_name = os.path.join(self.root_dir, 
                            '_'.join(self.img_labels.iloc[idx, 0].split('_')[0:3]),
                            self.img_labels.iloc[idx, 0].split('_')[-1])
    image = io.imread(img_name)
    sample = {'image':image}

    if self.transforms:
      sample = self.transform(sample)
    
    return sample


In [48]:
dataset = CustomImageDataset('DL_info.csv', 'zip_file/Images_png')

In [49]:
print(len(dataset))

32735


In [50]:
for i in dataset:
  print(i) ### i currently stores the image only
  break

{'image': array([[29744, 29744, 29744, ..., 29744, 29744, 29744],
       [29744, 29744, 29744, ..., 29744, 29744, 29744],
       [29744, 29744, 29744, ..., 29744, 29744, 29744],
       ...,
       [29744, 29744, 29744, ..., 29744, 29744, 29744],
       [29744, 29744, 29744, ..., 29744, 29744, 29744],
       [29744, 29744, 29744, ..., 29744, 29744, 29744]], dtype=uint16)}


Creating dataloader

Downloading SWIN IR model



curl 7.58.0 (x86_64-pc-linux-gnu) libcurl/7.58.0 OpenSSL/1.1.1 zlib/1.2.11 libidn2/2.0.4 libpsl/0.19.1 (+libidn2/2.0.4) nghttp2/1.30.0 librtmp/2.3
Release-Date: 2018-01-24
Protocols: dict file ftp ftps gopher http https imap imaps ldap ldaps pop3 pop3s rtmp rtsp smb smbs smtp smtps telnet tftp 
Features: AsynchDNS IDN IPv6 Largefile GSS-API Kerberos SPNEGO NTLM NTLM_WB SSL libz TLS-SRP HTTP2 UnixSockets HTTPS-proxy PSL 


In [ ]:
!wget 'https://raw.githubusercontent.com/cszn/KAIR/06bd194f4cd0da6f4159c058b6fb5b9be689c18a/main_train_psnr.py'
!wget 'https://raw.githubusercontent.com/cszn/KAIR/master/utils/utils_logger.py'
!wget 'https://raw.githubusercontent.com/cszn/KAIR/master/utils/utils_image.py'
!wget 'https://raw.githubusercontent.com/cszn/KAIR/master/utils/utils_option.py'
!wget 'https://raw.githubusercontent.com/cszn/KAIR/master/utils/utils_dist.py'
!wget 'https://raw.githubusercontent.com/cszn/KAIR/master/data/select_dataset.py'
!wget 'https://raw.githubusercontent.com/cszn/KAIR/master/models/select_model.py'

In [ ]:
!wget 'https://raw.githubusercontent.com/cszn/KAIR/06bd194f4cd0da6f4159c058b6fb5b9be689c18a/main_train_psnr.py'

In [ ]:
!wget 'https://raw.githubusercontent.com/cszn/KAIR/master/options/swinir/train_swinir_denoising_gray.json'

In [ ]:
!python -m torch.distributed.launch --nproc_per_node=8 --master_port=1234 main_train_psnr.py --opt options/swinir/train_swinir_denoising_gray.json  --dist True

# Playing around

In [6]:
import csv
file = open('DL_info.csv')
csv_reader = csv.reader(file)
header = []
header = next(csv_reader)
header

['File_name',
 'Patient_index',
 'Study_index',
 'Series_ID',
 'Key_slice_index',
 'Measurement_coordinates',
 'Bounding_boxes',
 'Lesion_diameters_Pixel_',
 'Normalized_lesion_location',
 'Coarse_lesion_type',
 'Possibly_noisy',
 'Slice_range',
 'Spacing_mm_px_',
 'Image_size',
 'DICOM_windows',
 'Patient_gender',
 'Patient_age',
 'Train_Val_Test']

In [ ]:
## name of files with images is in the form --> 'zip_file/Images_png/File_name.split('_')[0:3]/File_name.split('_')[-1]'

In [13]:
for row in csv_reader:
  print(row)
  print(row[0])
  print('_'.join(row[0].split('_')[0:3]))
  break

['000002_01_01_162.png', '2', '1', '1', '162', '238.314, 261.228, 235.858, 268.594, 240.36, 265.729, 234.222, 262.865', '229.222, 256.228, 245.36, 273.594', '7.76388, 6.77335', '0.437715, 0.573812, 0.609054', '2', '0', '156, 168', '0.859375, 0.859375, 5', '512, 512', '-175, 275', 'F', '51', '2']
000002_01_01_162.png
000002_01_01


In [ ]:
for row in csv_reader:
  img = 'zip_file/Images_png' + '_'.join(row[0].split('_')[0:3]) + row[0].split('_')[-1]


Reading CSV with pandas

In [ ]:
import pandas as pd

reader = pd.read_csv('DL_info.csv')
reader

In [ ]:
for i in reader:
  print(i)

In [ ]:
print(reader.head())  

In [24]:
print(len(reader))
print(reader.shape) ## 32735 rows and 18 cols

32735
(32735, 18)


In [34]:
reader.iloc[1, 0]  ### Pretty important!!!

'000001_02_01_014.png'